# **BUILD CHATGPT KIND OF SYSTEM FROM SCRATCH -

BUT start from basics

---




In this example, you will learn how to create linear model, binary classification and character level prediction model without any fancy language transformer

## example of linear regression using pytorch ###

In [ ]:
import torch
import torch.nn as nn

# Simple linear model
model = nn.Linear(1, 1)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

x = torch.tensor([[1.0], [2.0], [3.0]])
y = torch.tensor([[2.0], [4.0], [6.0]])

for epoch in range(100):
    pred = model(x)
    loss = criterion(pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
model(torch.tensor([[4.0]]))

tensor([[7.4299]], grad_fn=<AddmmBackward0>)

## end of linear model example ###

## Example of binary classification *model*

In [ ]:
import torch
import torch.nn.functional as F

# Simulated output from a model for a single input
logits = torch.tensor([2.0, 0.5])  # Raw scores for [cat, dog]

# Apply softmax to convert logits to probabilities
probs = F.softmax(logits, dim=0)

print("Logits:", logits)
print("Probabilities:", probs)

Logits: tensor([2.0000, 0.5000])
Probabilities: tensor([0.8176, 0.1824])


## End of binary classification model

## example of language model - character level prediction model ##

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
text = "hello"
chars = sorted(list(set(text)))
vocab_size = len(chars)

print(chars)


['e', 'h', 'l', 'o']


In [ ]:
# Mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for ch,i in stoi.items() }

# Encode and decode functions
def encode(s): return [stoi[c] for c in s]
def decode(l): return ''.join([itos[i] for i in l])

In [ ]:
# "hello" → input: "h", "e", "l", "l" → target: "e", "l", "l", "o"
data = torch.tensor(encode(text), dtype=torch.long)
x = data[:-1].unsqueeze(0)  # shape (1, 4)
y = data[1:].unsqueeze(0)   # shape (1, 4)


In [ ]:
print(data)

tensor([1, 0, 2, 2, 3])


# BIGRAM MODEL

A bigram model is a type of statistical language model in natural language processing (NLP) that predicts the likelihood of a word based on the word that comes immediately before it. It’s a simple yet powerful way to capture short-range word dependencies in text.



Sentence: “I love programming languages” Bigrams: (“I”, “love”), (“love”, “programming”), (“programming”, “languages”) Probability: $
𝑃
(
"
𝐼
"
)
×
𝑃
(
"
𝑙
𝑜
𝑣
𝑒
"
∣
"
𝐼
"
)
×
𝑃
(
"
𝑝
𝑟
𝑜
𝑔
𝑟
𝑎
𝑚
𝑚
𝑖
𝑛
𝑔
"
∣
"
𝑙
𝑜
𝑣
𝑒
"
)
×
𝑃
(
"
𝑙
𝑎
𝑛
𝑔
𝑢
𝑎
𝑔
𝑒
𝑠
"
∣
"
𝑝
𝑟
𝑜
𝑔
𝑟
𝑎
𝑚
𝑚
𝑖
𝑛
𝑔
"
)
$

self.token_embedding_table(idx) returns a tensor of shape (B, T, C):

B = batch size
T = sequence length
C = number of classes or vocabulary size (i.e., the dimension of the embedding)

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)  # (B,T,C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            print("Logits:", logits)
            print("targets:", targets)
            loss = F.cross_entropy(logits, targets)
        return logits, loss


In [ ]:
model = BigramLanguageModel(vocab_size)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)

# Ensure inputs are of type LongTensor
x = x.long()
y = y.long()

for step in range(100):
    logits, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}, Loss: {loss.item():.4f}")



Logits: tensor([[ 0.5382, -0.7925, -2.6604,  0.4119],
        [-0.9059, -0.7910,  0.0617, -0.5902],
        [ 0.3625,  1.0737,  1.6481, -0.1337],
        [ 0.3625,  1.0737,  1.6481, -0.1337]], grad_fn=<ViewBackward0>)
targets: tensor([0, 2, 2, 3])
Step 0, Loss: 1.2007
Logits: tensor([[ 0.5482, -0.8024, -2.6701,  0.4018],
        [-0.9158, -0.8009,  0.0717, -0.6001],
        [ 0.3524,  1.0635,  1.6580, -0.1237],
        [ 0.3524,  1.0635,  1.6580, -0.1237]], grad_fn=<ViewBackward0>)
targets: tensor([0, 2, 2, 3])
Logits: tensor([[ 0.5581, -0.8123, -2.6798,  0.3918],
        [-0.9257, -0.8108,  0.0817, -0.6101],
        [ 0.3424,  1.0534,  1.6567, -0.1137],
        [ 0.3424,  1.0534,  1.6567, -0.1137]], grad_fn=<ViewBackward0>)
targets: tensor([0, 2, 2, 3])
Logits: tensor([[ 0.5681, -0.8222, -2.6896,  0.3818],
        [-0.9356, -0.8207,  0.0917, -0.6200],
        [ 0.3324,  1.0433,  1.6510, -0.1037],
        [ 0.3324,  1.0433,  1.6510, -0.1037]], grad_fn=<ViewBackward0>)
targets: tensor([

In [ ]:
context = torch.tensor([[stoi['h']]], dtype=torch.long)

generated = []
for _ in range(10):
    logits, _ = model(context)
    probs = F.softmax(logits[:, -1, :], dim=-1)
    next_token = torch.multinomial(probs, num_samples=1)
    generated.append(next_token.item())
    context = next_token

print("Generated:", decode(generated))


Generated: elhelhelhe
